In [2]:
#settings
import math
import random
import copy
import string
import os
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import sys 
sys.path.append("..") 
from algebra import GaloisField as gf
from algebra import Tropical as tr

parameters={
    "row": 3, #向量x的行
    "column": 1, #向量x的列
    "times": 100, #迭代次数
    "random_ranges": 23, #随机赋值范围
    "loop_lenth": 50, #查找不定点的循环长度
    "gf_n": 23, #有限域
    "A": [[0,2,4], [6,1,7],[5,1,3]], #确定的矩阵与向量
    "x": [[9], [1],[2]],
    "save_path": "data"
}
# Suggest that random_ranges and gf_n have the same value, And a value of random_ranges less than gf_n is meaningless.

In [ ]:
class infoManager(): #管理生成数据的文件读写器
    
    def __init__(self, save_path) -> None:
        self.file_path = save_path #parameters["save_path"]
        if(not os.path.exists(self.file_path)):
            os.makedirs(self.file_path)
        self.target_path = self.file_path+"/"+"infoManager.txt"
        self.info = [0, 0]
        try:
            f = open(self.target_path, 'a')
            f.close()
        except IOError:
            self.info[0] = 0 #record the order of finding
            self.info[1] = 0 #record the order of analysis
            with open(self.target_path, 'w') as infoMng:
                infoMng.write("find_fixed_points_order: {0}\n".format(str(self.info[0])))
                infoMng.write("analysis_fixed_points_order: {0}".format(str(self.info[1])))
                
        with open(self.target_path, 'r') as infoMng:
            info = infoMng.readlines()
            if len(info) > 1:
                self.info = [int(info[0].strip().split(':')[1]), int(info[1].strip().split(':')[1])]
            else:
                self.info[0] = 0
                self.info[1] = 0
                with open(self.target_path, 'w') as infoMng:
                    infoMng.write("find_fixed_points_order: {0}\n".format(str(self.info[0])))
                    infoMng.write("analysis_fixed_points_order: {0}".format(str(self.info[1])))

   
    def inc_find(self):
        self.info[0] += 1
        with open(self.target_path, 'w') as infoMng:
            infoMng.write("find_fixed_points_order: {0}\n".format(str(self.info[0])))
            infoMng.write("analysis_fixed_points_order: {0}".format(str(self.info[1])))
    
    def inc_analysis(self):
        self.info[1] += 1
        with open(self.target_path, 'w') as infoMng:
            infoMng.write("find_fixed_points_order: {0}\n".format(str(self.info[0])))
            infoMng.write("analysis_fixed_points_order: {0}".format(str(self.info[1])))

    def clear(self):
        self.info = [0, 0]
        with open(self.target_path, 'w') as infoMng:
            infoMng.write("find_fixed_points_order: {0}\n".format(str(self.info[0])))
            infoMng.write("analysis_fixed_points_order: {0}".format(str(self.info[1])))
            
    def total_clear(self):

        def delete_files(file_path):
            if not os.path.exists(file_path):
                print("Folder not exists.")
                return
            files = os.listdir(file_path)
            os.chdir(file_path)
            for file in files:
                os.remove(file)
            os.chdir("..")
            print('Files deleted successfully.')

        self.info = [0, 0]
        with open(self.target_path, 'w') as infoMng:
            infoMng.write("find_fixed_points_order: {0}\n".format(0))
            infoMng.write("analysis_fixed_points_order: {0}".format(0))
        delete_files(self.file_path + '/Analysis')
        delete_files(self.file_path + '/ProcessDetail')



    @classmethod
    def instance(cls, *args, **kwargs):
        if not hasattr(infoManager, "_instance"):
            infoManager._instance = infoManager(*args, **kwargs)
        return infoManager._instance


In [ ]:
random_ranges = parameters["random_ranges"]
gf_n = parameters["gf_n"]
gf.Num.set_card(gf_n)

def set_random_tropical_num_in_GF(): # generate random number in tropical field
    return tr.Num(gf.Num(random.randint(0, random_ranges - 1)))

def set_random_num_in_GF(): # generate random number in GF field
    return gf.Num(random.randint(0, random_ranges - 1))

def set_np_matrix(i, j, f_getnum = lambda: 0):# generate random matrix
    if j > 0 and i > 0:
        b = [([0] * j)for line in range(i)]
        for ii in range(i):
            for jj in range(j):
                b[ii][jj] = f_getnum()
    b = np.array(b)
    return b

def convertTroArray(matrix_A: np.array) -> np.array:
    for cnti, i in enumerate(matrix_A):
        for cntj, j in enumerate(i):
            matrix_A[cnti][cntj] = tr.Num(gf.Num(j))
    matrix_A = np.array(matrix_A)
    return matrix_A


In [ ]:
def get_random_str():
    seed = "1234567890abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"
    sa = []
    for i in range(6):
        sa.append(random.choice(seed))
        salt = ''.join(sa)
    return salt

In [ ]:
def tropical_proj_after_affine(matrix_A, column_vector_x):
    column_vector_x = np.dot(matrix_A, column_vector_x)  # x = Ax
    minx = min(column_vector_x[0:][0])
    for elem in column_vector_x:  # x = proj(X)
        elem[0] = elem[0] / minx # division in tropical fields is equal to minus in fields of real numberss
    return column_vector_x


In [ ]:
def find_fixed_vector(matrix_A, column_vector_x, max_loop_lenth, iter_times, save_path = None, range_Num = -1, gf_num = -1, is_print = True): #if save_path is None, it will not save any process information
  isFound = False # is fixed points found
  vector_list = [] # record all the vectors
  loop_list = [] # record cyclic vectors
  loop_index = -1 # after how many multiplications fixed points take to appear
  loop_lenth = -1 # the length of time taken by one cycle 
  origin_vector_x = copy.deepcopy(column_vector_x)
  for iter in range(iter_times): #calculate
    record_vector_x=copy.deepcopy(column_vector_x)
    vector_list.append(record_vector_x)
    
    for index in range(max(0, len(vector_list) - max_loop_lenth), len(vector_list) - 1): # examine if fixed points take to appear
      if np.array_equal(column_vector_x, vector_list[index]) :
        isFound = True
        loop_index = index
      if(isFound): 
        loop_list.append(vector_list[index])

    if(isFound): 
      loop_lenth = len(loop_list)
      break

    column_vector_x = tropical_proj_after_affine(matrix_A, column_vector_x)
    
  # print results
  if is_print is True:
    print("Matrix A:\n{0}\n".format(matrix_A))
    print("Origin vector:\n{0}\n".format(origin_vector_x))
    print("\nThe value range is: 0 - {0}\n".format(range_Num))
    print("The GF is GF({0})\n".format(gf_num))
    print("==========================\n\n")
    if(isFound):
      print("After {0} times calculate, the fixed points take to appear.\n".format(loop_index))
      print("The lenth of cycle is: {0}\n".format(loop_lenth))
      print("\nCycle Vectors List:\n")
      for cnt, elem in enumerate(loop_list):
        print("{0}: {1}\n\n".format(cnt, elem))
    else:
      print("The fixed points are not found.\n")
  # save results
  if save_path is not None:
    orderManager = infoManager.instance(save_path)
    orderManager.inc_find()
    if (not os.path.exists(save_path + "/ProcessDetail")):
      os.makedirs(save_path + "/ProcessDetail")
    order = "000000" + str(orderManager.info[0])
    order = order[-6:]
    file_path = save_path + "/ProcessDetail/" + "Process_" + order +".txt" # parameters["save_path"]+"/"+"Process_"+order+".txt"
    with open(file_path, 'w') as file:
      file.write("Matrix A:\n{0}\n".format(matrix_A))
      file.write("Origin vector:\n{0}\n".format(origin_vector_x))
      file.write("\nThe value range is: 0 - {0}\n".format(range_Num))
      file.write("The GF is GF({0})\n".format(gf_num))
      file.write("==========================\n\n")
      if(isFound):
        file.write("After {0} times calculate, the fixed points take to appear.\n".format(loop_index))
        file.write("The lenth of cycle is: {0}\n".format(loop_lenth))
        file.write("\nCycle Vectors List:\n")
        for cnt, elem in enumerate(loop_list):
          file.write("{0}: {1}\n\n".format(cnt, elem))
      else:
        file.write("The fixed points are not found.\n")
      file.write("==========================\n")
      file.write("Vector list:\n")
      for cnt, elem in enumerate(vector_list):
        file.write("{0}: {1}\n\n".format(cnt, elem))

  return loop_list, loop_index, origin_vector_x
      



In [ ]:
class Analysis_Tropical_Matrix():

    def __init__(self, matrix_A, max_loop_lenth, iter_times, range_Num, gf_num, save_path) -> None:
        self.fileManager = infoManager.instance(save_path)
        self.matrixA = matrix_A
        self.max_loop_lenth = max_loop_lenth
        self.iter_times = iter_times
        self.save_path = save_path
        self.range_Num = range_Num
        self.gf_num = gf_num

    def analysis_of_given_matrix(self, given_vectors = None, high_proportion = 1, isDetailed = False):    
        self.fileManager.inc_analysis()
        analysisSet_list = []
        count_list = []
        # define empty loop set
        analysisSet_list.append([])
        count_list.append(0)
        # root folder
        if(not os.path.exists(self.save_path + "/Analysis")):
            os.makedirs(self.save_path + "/Analysis")
        # generate name
        order = "000000" + str(self.fileManager.info[1])
        order = order[-6:]  
        target_path = self.save_path + "/Analysis/" + "Analysis_" + order
        if(not os.path.exists(target_path)):
            os.makedirs(target_path)
        def is_loop_equal(listA, listB):    
            if listA == [] and listB == []:
                return True
            elif listA == [] or listB == []:
                return False
            else:
                itemfirst = listA[0]
                for elem in listB:
                    if elem == itemfirst:
                        return True
                return False
        def analyse_loop(column_vector_x: np.array):
            loop_list, loop_index, origin_vector_x = find_fixed_vector(self.matrixA, column_vector_x, self.max_loop_lenth, self.iter_times, save_path = None, is_print = False)
            loop_list = [tuple(item.reshape(-1)) for item in loop_list]
            find_flag = 0
            for cnt, elem in enumerate(analysisSet_list):
                if is_loop_equal(elem, loop_list):                    
                    count_list[cnt] += 1
                    find_flag = 1
                    break
            if find_flag == 0:
                analysisSet_list.append(loop_list)
                count_list.append(1)
                
            if isDetailed:
                with open(target_path + "/Detail.txt", 'a') as file:
                    file.write("==================\n")
                    file.write("Origin vector:\n{0}\n".format(origin_vector_x))
                    file.write("After {0} times calculate:\n".format(loop_index))
                    file.write("Loop list is type {0}\n".format(analysisSet_list.index(loop_list)))

        def dfs_vector(column_vector_x: np.array, rangeNum, searchNum = 0):
            vheight, vwidth = np.shape(column_vector_x)
            if searchNum == vheight:
                #print(column_vector_x)
                analyse_loop(column_vector_x)
                return            
            
            for iterNum in range(rangeNum):
                column_vector_x[searchNum][0] += iterNum
                dfs_vector(column_vector_x, rangeNum, searchNum + 1)
                column_vector_x[searchNum][0] -= iterNum

        # main phase
        mheight, mwidth = np.shape(self.matrixA)
        zero_vector_x = set_np_matrix(mheight, 1, lambda: tr.Num(gf.Num(0)))
        if given_vectors != None:
            for elem in given_vectors:
                analyse_loop(elem)
        else:
            dfs_vector(zero_vector_x, self.range_Num, 0)
            
        totalCount = sum(count_list)
        typeCount = len(analysisSet_list)
        
        # save file
        with open(target_path + "/Infomation.txt", 'a') as file:
            file.write("Total calculation times: {0}\n".format(totalCount))
            file.write("Matrix A:\n{0}\n".format(self.matrixA))
            file.write("\nThe value range is: 0 - {0}\n".format(self.range_Num))
            file.write("The GF is GF({0})\n".format(self.gf_num))
            file.write("\nThe total number of types: {0}\n".format(typeCount))
            file.write("Following loop types are high proportion(>{0}):\n".format(high_proportion))
            for cnt, elem in enumerate(analysisSet_list):
                if (count_list[cnt] / totalCount * 100) > high_proportion :
                    file.write("Type {0} with proportion {1:.6f}%\n".format(cnt, (count_list[cnt] / totalCount * 100)))
            file.write("\nNotice: All column vectors below will be represented as tuples, but they are still column vectors in fact.\n")
            for cnt, elem in enumerate(analysisSet_list):
                file.write("==================\n\n")
                file.write("Loop Type ==> {0}\n".format(cnt))
                file.write("The lenth of cycle is: {0}\n".format(len(elem)))
                file.write("\nCycle Vectors List:\n")
                for cnt1, elem1 in enumerate(elem):
                    file.write("{0}: {1}\n\n".format(cnt1, elem1))
                file.write("This Type contains {0} different start vectors, proportion {1:.6f}%\n".format(count_list[cnt], (count_list[cnt] / totalCount * 100)))
        
        # print type proportion curve image, take a type as one unit
        lenth_list = {}
        for cnt, elem in enumerate(analysisSet_list):
            elem_len = len(elem)
            if elem_len not in lenth_list:
                lenth_list.update({elem_len: 1})
            else:
                lenth_list[elem_len] += 1
        lenth_list = dict(sorted(lenth_list.items(), key = lambda x: x[0]))
        plt.title("loop lenth analysis 2", fontsize=20)
        plt.xlabel("loop lenth", fontsize=16)
        plt.ylabel("number", fontsize=16)
        #plt.rcParams['axes.facecolor'] = 'white'
        plt.plot([elem for elem in lenth_list.keys()], [elem for elem in lenth_list.values()], color = "red", label = "unit type") 
        [plt.text(elem, lenth_list[elem], '{0},{1},{2:.2f}%'.format(elem, lenth_list[elem], lenth_list[elem] / typeCount * 100), fontdict={'fontsize': 14}) for elem in lenth_list]
        plt.legend()
        plt.grid()
        plt.savefig(target_path + "/loop lenth 2.png")
        plt.show()

        # print number proportion curve image 
        lenth_list = {}
        for cnt, elem in enumerate(analysisSet_list):
            elem_len = len(elem)
            if elem_len not in lenth_list:
                lenth_list.update({elem_len: count_list[cnt]})
            else:
                lenth_list[elem_len] += count_list[cnt]
        lenth_list = dict(sorted(lenth_list.items(), key = lambda x: x[0]))
        plt.title("loop lenth analysis 1", fontsize=20)
        plt.xlabel("loop lenth", fontsize=16)
        plt.ylabel("number", fontsize=16)
        #plt.rcParams['axes.facecolor'] = 'white'
        plt.plot([elem for elem in lenth_list.keys()], [elem for elem in lenth_list.values()], color = "red", label = "amount of type") 
        [plt.text(elem, lenth_list[elem], '{0},{1},{2:.2f}%'.format(elem, lenth_list[elem], lenth_list[elem] / totalCount * 100), fontdict={'fontsize': 14}) for elem in lenth_list]
        plt.legend()
        plt.grid()
        plt.savefig(target_path + "/loop lenth 1.png")
        plt.show()

    @classmethod
    def instance(cls, *args, **kwargs):
        if not hasattr(Analysis_Tropical_Matrix, "_instance"):
            Analysis_Tropical_Matrix._instance = Analysis_Tropical_Matrix(*args, **kwargs)
        return Analysis_Tropical_Matrix._instance

In [ ]:
# make a random calculation
row = parameters["row"] 
column = parameters["column"]
times = parameters["times"]
random_ranges = parameters["random_ranges"]
loop_lenth = parameters["loop_lenth"]
gf_n = parameters["gf_n"]
save_path = parameters["save_path"]
certain_A = parameters["A"]
certain_x = parameters["x"]
gf.Num.set_card(gf_n)

matrix_A = set_np_matrix(row, row, set_random_tropical_num_in_GF)
column_vector_x = set_np_matrix(row, column, set_random_tropical_num_in_GF)
# matrix_A = certain_A
# column_vector_x = certain_x

find_fixed_vector(matrix_A, column_vector_x, loop_lenth, times, save_path, random_ranges, gf_n)
#loop_list, loop_index, origin_vector_x = find_fixed_vector(matrix_A, column_vector_x, loop_lenth, times, None, False)


In [ ]:
row = parameters["row"] 
column = parameters["column"]
times = parameters["times"]
random_ranges = parameters["random_ranges"]
loop_lenth = parameters["loop_lenth"]
gf_n = parameters["gf_n"]
save_path = parameters["save_path"]
certain_A = parameters["A"]
certain_x = parameters["x"]
gf.Num.set_card(gf_n)

matrix_A = set_np_matrix(row, row, set_random_tropical_num_in_GF)
#matrix_A = convertTroArray(certain_A, gf_n)

Analysis_Tropical_Matrix(matrix_A, loop_lenth, times, random_ranges, gf_n, save_path).analysis_of_given_matrix(high_proportion = 5)

'''
for i in range(row):
    matrix_A = set_np_matrix(row, row, lambda: tr.Num(gf.Num(0)))
    matrix_A[i][i] = tr.Num(gf.Num(1))
    Analysis_Tropical_Matrix(matrix_A, loop_lenth, times, random_ranges, gf_n, save_path).analysis_of_given_matrix(5)
'''


In [ ]:
#infoManager.instance(save_path).total_clear()